In [32]:
import numpy as np
import random
import pandas as pd
from sklearn.preprocessing import MinMaxScaler, LabelEncoder, OneHotEncoder

In [2]:
df = pd.read_csv('Churn_Modelling.csv')

In [3]:
df.shape

(10000, 14)

In [4]:
df.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [5]:
label = LabelEncoder()
df['Gender'] = label.fit_transform(df['Gender'])

In [7]:
# df.head()

In [8]:
df['Geography'] = label.fit_transform(df['Geography'])

In [9]:
df.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,0,0,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,2,0,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,0,0,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,0,0,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,2,0,43,2,125510.82,1,1,1,79084.10,0


In [12]:
geog = df['Geography'].values

In [13]:
onehot = OneHotEncoder()
geog = onehot.fit_transform(geog.reshape(-1,1))

In [14]:
geog

<10000x3 sparse matrix of type '<class 'numpy.float64'>'
	with 10000 stored elements in Compressed Sparse Row format>

In [15]:
geog = geog.toarray()

In [16]:
geog

array([[1., 0., 0.],
       [0., 0., 1.],
       [1., 0., 0.],
       ...,
       [1., 0., 0.],
       [0., 1., 0.],
       [1., 0., 0.]])

In [17]:
del df['Geography']

In [22]:
X = df.iloc[:, 3:].values

In [23]:
X = np.c_[X,geog]

In [24]:
X.shape

(10000, 13)

In [25]:
minmax = MinMaxScaler()
X = minmax.fit_transform(X)

In [33]:
def cross_validation(dataset, k=5):
    X_copy = list(dataset)
    fold_size = len(X) // 5
    folds = []
    for i in range(k):
        fold = []
        while len(fold) < fold_size:
            index = random.randrange(0, len(X_copy))
            fold.append(X_copy.pop(index))
        folds.append(fold)
    return folds

In [34]:
def predict(row, coef):
    x = coef[0]
    for i in range(len(row)):
        x += coef[i + 1] * row[i]
    return 1 / (1 + math.exp(-x))

In [35]:
def accuracy(actual, prediction):
    count = 0
    for i in range(len(actual)):
        if actual[i] == prediction[i]:
            count += 1
    return count / len(actual) * 100

In [38]:
def gradientDescent(x_train, y_train, epochs, alpha):
    coef = np.zeros(x_train.shape[1] + 1)
    n = len(x_train)
    for i in range(epochs):
        for j in range(n):
            pred = predict(x_train[j], coef)
            error = pred - y_train[j]
            coef[0] = coef[0] - alpha * (1/n) * np.sum(error)
            for k in range(len(x_train[0] - 1)):
                coef[k + 1] = coef[0] - alpha * (1/n) * np.dot(x_train[j][k], error)
        print("{} Epochs completed...")
    return coef

In [36]:
def logistic(x_train, y_train, x_test, y_test, epochs, alpha):
    coef = gradientDescent(x_train, y_train, epochs, alpha)
    predictions = []
    for row in x_test:
        pred = predict(row, coef)
        predictions.append(round(pred))
    acc =  accuracy(y_test, predictions)
    return acc

In [37]:
def evaluate(X, epochs, alpha):
    folds = cross_validation(X)
    scores = []
    for i in range(len(folds)):
    
        acc = logistic(x_train, y_train, x_test, y_test, epochs, alpha)
        scores.append(acc)
    
    return scores

In [ ]:
epochs = 50
aloha = 0.01
evaluate(X, epochs, alpha)